# Sales Data Cleaning - CP610 Deliverable #2

**Purpose**: Clean and prepare sales transaction data for analysis  
**Input**: `../datasources/Sales_v4.csv`  
**Output**: `../output_data/Sales_cleaned.csv`  
**Strategy**: See `../CLEANING_STRATEGY.md` for detailed methodology

**Key Cleaning Approach**: **Transaction_ID + Date Composite Key**
- 339 duplicate Transaction IDs resolved by creating composite key: Transaction_ID + Date
- Original Transaction ID preserved in `Original_Transaction_ID` column
- All 25,000 transactions retained with 100% unique identifiers (no artificial suffixes needed)
- Semantically meaningful: reflects business reality that Transaction IDs are unique within a date

---

## Table of Contents
1. [Load & Initial Exploration](#1)
2. [Critical Issue: Duplicate Transaction IDs](#2)
3. [Data Quality Assessment](#3)
4. [Data Cleaning Operations](#4)
5. [Feature Engineering](#5)
6. [Data Validation](#6)
7. [Export Cleaned Data](#7)
8. [Comprehensive Quality Report](#8)

---
## 1. Load & Initial Exploration

In [10]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load sales data
df = pd.read_csv('../datasources/Sales_v4.csv')

print("="*60)
print("SALES DATA LOADED")
print("="*60)
print(f"Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("="*60)

# Display info and sample
print("\nDataset Info:")
df.info()
print("\nFirst 10 rows:")
display(df.head(10))
print("\nSummary Statistics:")
display(df.describe())

SALES DATA LOADED
Dataset shape: 25,000 rows × 13 columns
Memory usage: 12.24 MB

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Transaction ID      25000 non-null  object 
 1   Customer ID         25000 non-null  object 
 2   Location            25000 non-null  object 
 3   Payment Method      25000 non-null  object 
 4   Category            25000 non-null  object 
 5   Item                25000 non-null  object 
 6   Quantity            25000 non-null  int64  
 7   Unit Price          25000 non-null  float64
 8   Unit_Cost           25000 non-null  float64
 9   Pre_Discount_Total  25000 non-null  float64
 10  Discount_Rate       25000 non-null  float64
 11  Total Spent         25000 non-null  float64
 12  Date                25000 non-null  object 
dtypes: float64(5), int64(1), object(7)
memory usage: 2.5+ M

,Transaction ID,Customer ID,Location,Payment Method,Category,Item,Quantity,Unit Price,Unit_Cost,Pre_Discount_Total,Discount_Rate,Total Spent,Date
0,TXN_490235,CUST_0175,In-store,Cash,Beverages,Energy Drink,63,2.99,1.62,188.37,0.044,180.08,2024-01-25
1,TXN_159495,CUST_0516,Online,Credit Card,Beverages,Bottled Water,38,1.05,0.55,39.90,0.000,39.90,2024-12-27
2,TXN_966263,CUST_0311,In-store,Cash,Beverages,Bottled Water,59,1.05,0.85,61.95,0.000,61.95,2023-08-20
3,TXN_703561,CUST_0322,In-store,Credit Card,Beverages,Tea Pack,50,2.86,1.83,143.00,0.041,137.14,2025-08-20
4,TXN_372997,CUST_0579,In-store,Credit Card,Beverages,Bottled Water,49,1.05,0.91,51.45,0.000,51.45,2024-08-07
5,TXN_158116,CUST_0135,In-store,Cash,Beverages,Energy Drink,12,2.99,2.15,35.88,0.000,35.88,2023-09-04
6,TXN_847823,CUST_0055,Online,Digital Wallet,Beverages,Energy Drink,49,2.99,1.96,146.51,0.033,141.68,2023-09-21
7,TXN_970778,CUST_0976,In-store,Cash,Beverages,Energy Drink,41,2.99,1.58,122.59,0.044,117.20,2025-08-15
8,TXN_902529,CUST_0543,In-store,Credit Card,Beverages,Soft Drink,45,2.20,1.51,99.00,0.000,99.00,2024-11-18
9,TXN_951291,CUST_0586,In-store,Credit Card,Beverages,Energy Drink,31,2.99,1.83,92.69,0.000,92.69,2024-12-02



Summary Statistics:


,Quantity,Unit Price,Unit_Cost,Pre_Discount_Total,Discount_Rate,Total Spent
count,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,10.387760,159.926181,112.375159,553.573879,0.027501,501.424512
std,11.310347,437.743173,312.600882,1644.115697,0.034406,1434.715322
min,1.000000,1.050000,0.530000,2.720000,0.000000,2.720000
25%,3.000000,5.160000,3.220000,42.675000,0.000000,42.675000
50%,7.000000,11.770000,8.105000,92.690000,0.000000,92.690000
75%,13.000000,53.610000,38.857500,254.700000,0.046000,243.647500
max,134.000000,2208.990000,1983.260000,29047.800000,0.187000,25039.200000


---
## 2. Critical Issue: Duplicate Transaction IDs  
### Resolving Using Transaction_ID + Date Composite Key

In [11]:
# Step 1: Identify duplicate Transaction IDs and analyze patterns
print("="*60)
print("DUPLICATE TRANSACTION ID ANALYSIS")
print("="*60)

total_rows = len(df)
unique_ids = df['Transaction ID'].nunique()
duplicate_count = total_rows - unique_ids

print(f"Total transactions: {total_rows:,}")
print(f"Unique Transaction IDs: {unique_ids:,}")
print(f"Duplicate Transaction IDs: {duplicate_count:,}")
print(f"Rows affected by duplicates: {df.duplicated(subset=['Transaction ID'], keep=False).sum():,}")

# Convert Date to datetime for composite key analysis
df['Date'] = pd.to_datetime(df['Date'])

# Check if Transaction_ID + Date creates uniqueness
df['Date_Component'] = df['Date'].dt.strftime('%Y%m%d')
df['Transaction_ID_Date'] = df['Transaction ID'] + '_' + df['Date_Component']
unique_composite = df['Transaction_ID_Date'].nunique()

print(f"\n🔍 Testing Transaction_ID + Date Composite Key:")
print(f"Unique Transaction_ID + Date combinations: {unique_composite:,}")
print(f"Achieves 100% uniqueness: {unique_composite == total_rows}")

if unique_composite == total_rows:
    print(f"✅ Transaction_ID + Date successfully resolves all {duplicate_count} duplicates!")
    print(f"✅ No artificial suffixes needed - using natural composite key")
else:
    print(f"⚠️ Still {total_rows - unique_composite} duplicates remaining")

# Store original Transaction ID for reference
df['Original_Transaction_ID'] = df['Transaction ID'].copy()

print(f"\nExample composite Transaction IDs:")
print(df[['Original_Transaction_ID', 'Date', 'Transaction_ID_Date']].head(5).to_string(index=False))

DUPLICATE TRANSACTION ID ANALYSIS
Total transactions: 25,000
Unique Transaction IDs: 24,661
Duplicate Transaction IDs: 339
Rows affected by duplicates: 673

🔍 Testing Transaction_ID + Date Composite Key:
Unique Transaction_ID + Date combinations: 25,000
Achieves 100% uniqueness: True
✅ Transaction_ID + Date successfully resolves all 339 duplicates!
✅ No artificial suffixes needed - using natural composite key

Example composite Transaction IDs:
Original_Transaction_ID       Date Transaction_ID_Date
             TXN_490235 2024-01-25 TXN_490235_20240125
             TXN_159495 2024-12-27 TXN_159495_20241227
             TXN_966263 2023-08-20 TXN_966263_20230820
             TXN_703561 2025-08-20 TXN_703561_20250820
             TXN_372997 2024-08-07 TXN_372997_20240807


In [12]:
# Step 2: Apply Transaction_ID + Date composite key
print("="*60)
print("CREATING UNIQUE TRANSACTION IDs WITH DATE COMPOSITE KEY")
print("="*60)

# Replace Transaction ID with the composite key
df['Transaction ID'] = df['Transaction_ID_Date'].copy()

# Verify uniqueness
print(f"✅ All Transaction IDs are now unique: {df['Transaction ID'].is_unique}")
print(f"Total unique Transaction IDs: {df['Transaction ID'].nunique():,}")

# Show examples
print("\n📋 Examples of new Transaction IDs:")
sample = df[['Original_Transaction_ID', 'Date', 'Transaction ID', 'Customer ID', 'Total Spent']].head(10)
display(sample)

print("\n🔍 Approach Summary:")
print("  • Original Transaction IDs preserved in 'Original_Transaction_ID' column")
print("  • New Transaction ID format: TXN_XXXXXX_YYYYMMDD")
print("  • Semantically meaningful: Transaction is unique within a date")
print("  • No artificial suffixes needed (_1, _2, etc.)")
print("  • 100% uniqueness achieved naturally")

CREATING UNIQUE TRANSACTION IDs WITH DATE COMPOSITE KEY
✅ All Transaction IDs are now unique: True
Total unique Transaction IDs: 25,000

📋 Examples of new Transaction IDs:


,Original_Transaction_ID,Date,Transaction ID,Customer ID,Total Spent
0,TXN_490235,2024-01-25,TXN_490235_20240125,CUST_0175,180.08
1,TXN_159495,2024-12-27,TXN_159495_20241227,CUST_0516,39.90
2,TXN_966263,2023-08-20,TXN_966263_20230820,CUST_0311,61.95
3,TXN_703561,2025-08-20,TXN_703561_20250820,CUST_0322,137.14
4,TXN_372997,2024-08-07,TXN_372997_20240807,CUST_0579,51.45
5,TXN_158116,2023-09-04,TXN_158116_20230904,CUST_0135,35.88
6,TXN_847823,2023-09-21,TXN_847823_20230921,CUST_0055,141.68
7,TXN_970778,2025-08-15,TXN_970778_20250815,CUST_0976,117.20
8,TXN_902529,2024-11-18,TXN_902529_20241118,CUST_0543,99.00
9,TXN_951291,2024-12-02,TXN_951291_20241202,CUST_0586,92.69



🔍 Approach Summary:
  • Original Transaction IDs preserved in 'Original_Transaction_ID' column
  • New Transaction ID format: TXN_XXXXXX_YYYYMMDD
  • Semantically meaningful: Transaction is unique within a date
  • No artificial suffixes needed (_1, _2, etc.)
  • 100% uniqueness achieved naturally


---
## 3. Data Quality Assessment

In [13]:
# Data quality assessment
TODAY = pd.Timestamp('2025-10-24')
df['has_future_date'] = df['Date'] > TODAY

# Load cleaned customer data for validation
customers = pd.read_csv('../output_data/Customers_cleaned.csv')
valid_customer_ids = set(customers['Customer ID'].unique())
df['has_invalid_customer'] = ~df['Customer ID'].isin(valid_customer_ids)

# Check mathematical consistency
df['Expected_Pre_Discount'] = df['Quantity'] * df['Unit Price']
df['Pre_Discount_Error'] = abs(df['Pre_Discount_Total'] - df['Expected_Pre_Discount']) > 0.01
df['Expected_Total'] = df['Pre_Discount_Total'] * (1 - df['Discount_Rate'])
df['Total_Error'] = abs(df['Total Spent'] - df['Expected_Total']) > 0.01

# Check for negative values and invalid discount rates
df['has_negative_qty'] = df['Quantity'] < 0
df['has_negative_price'] = df['Unit Price'] < 0
df['has_negative_cost'] = df['Unit_Cost'] < 0
df['has_negative_total'] = df['Total Spent'] < 0
df['invalid_discount'] = ~df['Discount_Rate'].between(0, 1)

# Print quality assessment
print("="*60)
print("DATA QUALITY ASSESSMENT")
print("="*60)
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Future dates: {df['has_future_date'].sum()}")
print(f"Invalid Customer IDs: {df['has_invalid_customer'].sum()}")
print(f"Pre-discount errors: {df['Pre_Discount_Error'].sum()}")
print(f"Total calculation errors: {df['Total_Error'].sum()}")
print(f"Negative quantities: {df['has_negative_qty'].sum()}")
print(f"Negative prices: {df['has_negative_price'].sum()}")
print(f"Negative totals: {df['has_negative_total'].sum()}")
print(f"Invalid discount rates: {df['invalid_discount'].sum()}")
print("="*60)

DATA QUALITY ASSESSMENT
Missing values: 0
Future dates: 111
Invalid Customer IDs: 0
Pre-discount errors: 0
Total calculation errors: 0
Negative quantities: 0
Negative prices: 0
Negative totals: 0
Invalid discount rates: 0


---
## 4. Data Cleaning Operations

In [14]:
# Create cleaned copy
df_clean = df.copy()
print(f"Starting with {len(df_clean):,} rows\n")

# Recalculate derived fields if errors found
if df_clean['Pre_Discount_Error'].sum() > 0:
    df_clean.loc[df_clean['Pre_Discount_Error'], 'Pre_Discount_Total'] = df_clean['Expected_Pre_Discount']
    print(f"✓ Fixed {df_clean['Pre_Discount_Error'].sum()} pre-discount calculation errors")

if df_clean['Total_Error'].sum() > 0:
    df_clean.loc[df_clean['Total_Error'], 'Total Spent'] = df_clean['Expected_Total']
    print(f"✓ Fixed {df_clean['Total_Error'].sum()} total spent calculation errors")

# Round to 2 decimal places
df_clean['Pre_Discount_Total'] = df_clean['Pre_Discount_Total'].round(2)
df_clean['Total Spent'] = df_clean['Total Spent'].round(2)

# Remove rows with critical errors (negative values)
before = len(df_clean)
df_clean = df_clean[df_clean['Quantity'] > 0]
df_clean = df_clean[df_clean['Unit Price'] > 0]
df_clean = df_clean[df_clean['Total Spent'] >= 0]
after = len(df_clean)
print(f"✓ Removed {before - after} rows with negative quantities/prices/totals")

# Standardize categorical fields
df_clean['Location'] = df_clean['Location'].str.strip().str.title()
df_clean['Payment Method'] = df_clean['Payment Method'].str.strip().str.title()
df_clean['Category'] = df_clean['Category'].str.strip().str.title()
df_clean['Item'] = df_clean['Item'].str.strip()
print(f"✓ Standardized categorical fields")

print(f"\nRows remaining: {len(df_clean):,}")

Starting with 25,000 rows

✓ Removed 0 rows with negative quantities/prices/totals
✓ Standardized categorical fields

Rows remaining: 25,000


---
## 5. Feature Engineering

In [15]:
# Extract date components
df_clean['Year'] = df_clean['Date'].dt.year
df_clean['Month'] = df_clean['Date'].dt.month
df_clean['Day'] = df_clean['Date'].dt.day
df_clean['Day_of_Week'] = df_clean['Date'].dt.dayofweek
df_clean['Day_Name'] = df_clean['Date'].dt.day_name()
df_clean['Quarter'] = df_clean['Date'].dt.quarter
df_clean['Week_of_Year'] = df_clean['Date'].dt.isocalendar().week

# Calculate profit metrics
df_clean['Profit_Per_Unit'] = df_clean['Unit Price'] - df_clean['Unit_Cost']
df_clean['Total_Profit'] = (df_clean['Profit_Per_Unit'] * df_clean['Quantity']).round(2)
df_clean['Profit_Margin'] = ((df_clean['Profit_Per_Unit'] / df_clean['Unit Price']) * 100).round(2)

# Create data quality summary flag
def create_quality_flag(row):
    flags = []
    if row.get('has_future_date', False):
        flags.append('FUTURE_DATE')
    if row.get('has_invalid_customer', False):
        flags.append('INVALID_CUSTOMER')
    return '|'.join(flags) if flags else 'OK'

df_clean['data_quality_flag'] = df_clean.apply(create_quality_flag, axis=1)

print("="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)
print("✓ Date components extracted (Year, Month, Day, Quarter, etc.)")
print("✓ Profit metrics calculated (Profit Per Unit, Total Profit, Profit Margin)")
print("✓ Data quality flags created")
print(f"\nTotal columns: {len(df_clean.columns)}")

FEATURE ENGINEERING COMPLETE
✓ Date components extracted (Year, Month, Day, Quarter, etc.)
✓ Profit metrics calculated (Profit Per Unit, Total Profit, Profit Margin)
✓ Data quality flags created

Total columns: 38


---
## 6. Data Validation

In [16]:
# Final validation checks
print("="*60)
print("FINAL VALIDATION CHECKS")
print("="*60)

# 1. Transaction ID uniqueness
print(f"✓ Transaction IDs unique: {df_clean['Transaction ID'].is_unique}")

# 2. No missing critical fields
critical_fields = ['Transaction ID', 'Customer ID', 'Date', 'Total Spent']
missing = df_clean[critical_fields].isnull().sum().sum()
print(f"✓ Missing values in critical fields: {missing}")

# 3. Date is datetime
print(f"✓ Date column is datetime: {pd.api.types.is_datetime64_any_dtype(df_clean['Date'])}")

# 4. Positive values
print(f"✓ All quantities positive: {(df_clean['Quantity'] > 0).all()}")
print(f"✓ All prices positive: {(df_clean['Unit Price'] > 0).all()}")
print(f"✓ All totals non-negative: {(df_clean['Total Spent'] >= 0).all()}")

# 5. Valid discount rates
print(f"✓ All discount rates valid (0-1): {df_clean['Discount_Rate'].between(0, 1).all()}")

print("\n" + "="*60)
print("✅ ALL VALIDATION CHECKS PASSED!")
print("="*60)

FINAL VALIDATION CHECKS
✓ Transaction IDs unique: True
✓ Missing values in critical fields: 0
✓ Date column is datetime: True
✓ All quantities positive: True
✓ All prices positive: True
✓ All totals non-negative: True
✓ All discount rates valid (0-1): True

✅ ALL VALIDATION CHECKS PASSED!


---
## 7. Export Cleaned Data

In [17]:
# Export cleaned data
os.makedirs('../output_data', exist_ok=True)
output_file = '../output_data/Sales_cleaned.csv'
df_clean.to_csv(output_file, index=False)

print("="*60)
print("CLEANED DATA EXPORTED")
print("="*60)
print(f"Output file: {output_file}")
print(f"Rows exported: {len(df_clean):,}")
print(f"Columns: {len(df_clean.columns)}")
print("="*60)

CLEANED DATA EXPORTED
Output file: ../output_data/Sales_cleaned.csv
Rows exported: 25,000
Columns: 38


---
## 8. Comprehensive Quality Report

In [18]:
# Generate comprehensive quality report
quality_report = {
    'Dataset': 'Sales Data',
    'Original_Rows': len(df),
    'Cleaned_Rows': len(df_clean),
    'Rows_Removed': len(df) - len(df_clean),
    'Percentage_Retained': f"{(len(df_clean) / len(df) * 100):.2f}%",
    'Original_Unique_Transaction_IDs': df['Original_Transaction_ID'].nunique(),
    'Duplicate_Transaction_IDs_Found': len(df) - df['Original_Transaction_ID'].nunique(),
    'Resolution_Method': 'Transaction_ID + Date Composite Key',
    'Final_Unique_Transaction_IDs': df_clean['Transaction ID'].nunique(),
    'Uniqueness_Achieved': '100% (Natural Composite Key)',
    'Transactions_With_Future_Dates': df['has_future_date'].sum(),
    'Date_Range': f"{df_clean['Date'].min()} to {df_clean['Date'].max()}",
    'Transactions_With_Invalid_Customer_IDs': df['has_invalid_customer'].sum(),
    'Missing_Values_Original': df.isnull().sum().sum(),
    'Missing_Values_Cleaned': df_clean.isnull().sum().sum(),
    'Pre_Discount_Calculation_Errors': df['Pre_Discount_Error'].sum(),
    'Total_Calculation_Errors': df['Total_Error'].sum(),
    'Total_Transactions': len(df_clean),
    'Unique_Customers': df_clean['Customer ID'].nunique(),
    'Total_Revenue': f"${df_clean['Total Spent'].sum():,.2f}",
    'Total_Profit': f"${df_clean['Total_Profit'].sum():,.2f}",
    'Average_Transaction_Value': f"${df_clean['Total Spent'].mean():.2f}",
    'Average_Profit_Margin': f"{df_clean['Profit_Margin'].mean():.2f}%"
}

# Print report
print("\n" + "="*60)
print("SALES DATA QUALITY REPORT")
print("="*60)
for key, value in quality_report.items():
    print(f"{key}: {value}")
print("="*60)

# Save report to file
report_file = '../output_data/Sales_Quality_Report.txt'
with open(report_file, 'w') as f:
    f.write("SALES DATA QUALITY REPORT\n")
    f.write("="*60 + "\n\n")
    for key, value in quality_report.items():
        f.write(f"{key}: {value}\n")
    f.write("\n" + "="*60 + "\n")
    f.write("\nCleaning completed successfully!\n")
    f.write("Cleaned data available at: output_data/Sales_cleaned.csv\n")

print(f"\n✅ Quality report saved to: {report_file}")
print("\n🎉 SALES DATA CLEANING COMPLETE!")


SALES DATA QUALITY REPORT
Dataset: Sales Data
Original_Rows: 25000
Cleaned_Rows: 25000
Rows_Removed: 0
Percentage_Retained: 100.00%
Original_Unique_Transaction_IDs: 24661
Duplicate_Transaction_IDs_Found: 339
Resolution_Method: Transaction_ID + Date Composite Key
Final_Unique_Transaction_IDs: 25000
Uniqueness_Achieved: 100% (Natural Composite Key)
Transactions_With_Future_Dates: 111
Date_Range: 2023-01-01 00:00:00 to 2025-10-28 00:00:00
Transactions_With_Invalid_Customer_IDs: 0
Missing_Values_Original: 0
Missing_Values_Cleaned: 0
Pre_Discount_Calculation_Errors: 0
Total_Calculation_Errors: 0
Total_Transactions: 25000
Unique_Customers: 1000
Total_Revenue: $12,535,612.80
Total_Profit: $4,094,554.28
Average_Transaction_Value: $501.42
Average_Profit_Margin: 29.83%

✅ Quality report saved to: ../output_data/Sales_Quality_Report.txt

🎉 SALES DATA CLEANING COMPLETE!
